In [2]:
import os
import re

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

from ab.imputation import median_imputer

DATA_DIR = "../data"

# Placeholder for group target
target_column = "death_5_Y"

ModuleNotFoundError: No module named 'stratifyhfml'

In [3]:
data = pd.read_csv(os.path.join(DATA_DIR, "processed", "classification.csv"))
data = data.dropna(axis=1, how="all")

# Separating features and target
X = data.drop(columns=[target_column])

y = data[target_column]

In [9]:
preprocessor = median_imputer

model_params = {
    'classifier__class_weight': 'balanced',
    'classifier__max_depth': 20,
    'classifier__min_samples_leaf': 2,
    'classifier__min_samples_split': 10,
    'classifier__n_estimators': 200,
}

model_params = {re.sub("^classifier__", "", k): v for k, v in model_params.items()}

clf = RandomForestClassifier(**model_params)

pipe = Pipeline(steps=[
    ("imputer", preprocessor),
    ("model", clf)
])

In [10]:
pipe.fit(X, y)

Pipeline(steps=[('imputer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000025DD44C4620>),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000025DD44C4650>)])),
                ('model',
                 RandomForestClassifier(class_weight='balanced', max_depth=20,
                                        min_samples_leaf=2,
                                        min_samples_split=10,
                                        n_estimators=200))])

In [11]:
import mlflow
import mlflow.sklearn

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("disease_progression_classification")

# Start an MLflow run
with mlflow.start_run() as run:
    # Log the model
    mlflow.sklearn.log_model(
        sk_model=pipe,
        name="disease_progression",
        registered_model_name="DiseaseProgression"  # Optional: Name in the model registry
    )

    # Log parameters or metrics if needed
    mlflow.log_param("model_type", "RandomForest")
    mlflow.log_param("preprocessing", "median_imputer")

2025/10/09 13:08:20 INFO mlflow.tracking.fluent: Experiment with name 'early_diagnosis_NT' does not exist. Creating a new experiment.
2025/10/09 13:08:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'EarlyDiagnosis' already exists. Creating a new version of this model...
2025/10/09 13:08:26 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: EarlyDiagnosis, version 3
Created version '3' of model 'EarlyDiagnosis'.


🏃 View run amusing-jay-123 at: http://127.0.0.1:5000/#/experiments/366571651235825243/runs/42c00ef221ef48538e51edbfb8ffc984
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/366571651235825243
